In [14]:
import warnings 
warnings.filterwarnings('ignore')

In [15]:
from crewai import Agent ,Task,Crew,LLM
llm= LLM(
    model="ollama/gemma3:4b",
    base_url="http://localhost:11434"
)

In [16]:
import os 
from dotenv import load_dotenv
load_dotenv()
serper_api_key = os.getenv("SERPER_API_KEY")

In [17]:
import yaml

### Load Agents , Tasks Yaml Files 

In [18]:
files = {
    'agents':'Config/agents.yaml',
    'tasks':'Config/tasks.yaml'
}

config={}
for config_type,file_path in files.items():
    with open(file_path,'r') as file:
        config[config_type]=yaml.safe_load(file)


agents_config = config['agents']
tasks_config = config['tasks']

### Create Pydantic Models for Structured Output 

In [19]:
from typing import List 
from litellm import Field
from pydantic import BaseModel

class TaskEstimate(BaseModel):
    task_name:str=Field(...,description="Name of the task")
    estimated_time_hours:float = Field(...,description="Estimated time to compelete the task in hours")
    required_resources:List[str]=Field(...,description="List of resources required to complate the task")

class Milestone(BaseModel):
    milestone_name:str = Field(...,description="Name of the milestone")
    tasks:list[str]= Field(...,description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks:List[TaskEstimate]= Field(...,description="List of tasks with their estimates")
    milestones:List[Milestone] = Field(...,description="List of project milestones")

### Create Agent , Task, Crew 

In [20]:
# Creating Agents
project_planning_agent = Agent(
    config=agents_config['project_planning_agent'],
    llm=llm
)
estimate_agent = Agent(
    config = agents_config['estimation_agent'],
    llm=llm
)
resource_allocation_agent = Agent(
    config = agents_config['resource_allocation_agent'],
    llm=llm
)

# Creating Tasks
task_breakdown = Task(
    config = tasks_config['task_breakdown'],
    agent=project_planning_agent
)
time_resource_estimate =Task(
    config = tasks_config['time_resource_estimation'],
    agent = estimate_agent
)
resource_allocation = Task(
    config = tasks_config['resource_allocation'],
    agent= resource_allocation_agent,
    output_pydantic=ProjectPlan
)

# Creating Crew 
crew = Crew(
    agents =[
        project_planning_agent,
        estimate_agent,
        resource_allocation_agent
    ],
    tasks=[
        task_breakdown,
        time_resource_estimate,
        resource_allocation
    ],
    verbose=True
)

In [21]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



### Kickoff Crew 

In [22]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2fd32f5c-401e-41b0-aa84-a67f1da993f4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Task: Carefully analyze the project_requirements for the Website project and break them down into individual   │
│  tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are      │
│  accounted for:                                                                                                 │
│                                                                                                                 │
│  - Create a responsive design that works well on desktop and mobile devices                                     │
│  - Implement a modern, visually appealing user interface with a clean look                                      │
│  - Develop a user-friendly navigation system with intuitive menu structure                                      │
│  - Include an "About Us" page highlighting the company's history and values                                     │
│  - Design a "Services" page showcasing the business's offerings with descriptions                               │
│  - Create a "Contact Us" page with a form and integrated map for communication                                  │
│  - Implement a blog section for sharing industry news and company updates                                       │
│  - Ensure fast loading times and optimize for search engines (SEO)                                              │
│  - Integrate social media links and sharing capabilities                                                        │
│  - Include a testimonials section to showcase customer feedback and build trust                                 │
│                                                                                                                 │
│                                                                                                                 │
│  Team members:                                                                                                  │
│                                                                                                                 │
│  - John Doe (Project Manager)                                                                                   │
│  - Jane Doe (Software Engineer)                                                                                 │
│  - Bob Smith (Designer)                                                                                         │
│  - Alice Johnson (QA Engineer)                                                                                  │
│  - Tom Brown (QA Engineer)                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Website Project Plan: Small Business Website**                                                               │
│                                                                                                                 │
│  **Project Manager:** John Doe                                                                                  │
│  **Team:** John Doe, Jane Doe, Bob Smith, Alice Johnson, Tom Brown                                              │
│                                                                                                                 │
│  **1. Project Initiation & Planning (Week 1)**                                                                  │
│                                                                                                                 │
│  *   **Task 1.1: Requirements Gathering & Confirmation** (John Doe - 2 days)                                    │
│      *   *Scope:* Detailed discussion with the client to confirm all requirements. Document final               │
│  specifications.                                                                                                │
│      *   *Timeline:* Day 1-2                                                                                    │
│      *   *Dependencies:* None                                                                                   │
│      *   *Deliverable:* Requirements Document v1                                                                │
│  *   **Task 1.2: Project Plan Finalization** (John Doe - 1 day)                                                 │
│      *   *Scope:* Formalize the project plan, including WBS, timelines, and resource allocation.                │
│      *   *Timeline:* Day 3                                                                                      │
│      *   *Dependencies:* Task 1.1                                                                               │
│      *   *Deliverable:* Project Plan v1                                                                         │
│                                                                                                                 │
│  **2. Design Phase (Weeks 2-4)**                                                                                │
│                                                                                                                 │
│  *   **Task 2.1: Wireframing & Mockups (Desktop)** (Bob Smith - 5 days)                                         │
│      *   *Scope:* Create low-fidelity wireframes for key pages (Homepage, Services, About Us, Contact Us).      │
│      *   *Timeline:* Week 2                                                                                     │
│      *   *Dependencies:* Task 1.2                                                                               │
│      *   *Deliverable:* Desktop Wireframes v1                                                                   │
│  *   **Task 2.2: Responsive Design Mockups** (Bob Smith - 7 days)                                               │
│      *   *Scope:* Develop high-fidelity mockups for all pages optimized for various screen sizes (desktop,      │
│  tablet, mobile).                                                                                               │
│      *   *Timeline:* Week 3                            

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 09988cc1-79fb-428e-8401-95aac765aae6                                                                     │
│  Agent: The Ultimate Project Planner                                                                            │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Estimation Analyst                                                                               │
│                                                                                                                 │
│  Task: Thoroughly evaluate each task in the Website project to estimate the time, resources, and effort         │
│  required. Use historical data, task complexity, and available resources to provide a realistic estimation for  │
│  each task.                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d57c3a7a-1a78-42e0-8085-e615884e9851                                                                     │
│  Agent: Expert Estimation Analyst                                                                               │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Task: Strategically allocate tasks for the Website project to team members based on their skills,              │
│  availability, and current workload. Ensure that each task is assigned to the most suitable team member and     │
│  that the workload is evenly distributed.                                                                       │
│                                                                                                                 │
│  Team members:                                                                                                  │
│                                                                                                                 │
│  - John Doe (Project Manager)                                                                                   │
│  - Jane Doe (Software Engineer)                                                                                 │
│  - Bob Smith (Designer)                                                                                         │
│  - Alice Johnson (QA Engineer)                                                                                  │
│  - Tom Brown (QA Engineer)                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Requirements Gathering & Confirmation",                                                    │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["John Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Project Plan Finalization",                                                                │
│        "estimated_time_hours": 1,                                                                               │
│        "required_resources": ["John Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Wireframing & Mockups (Desktop)",                                                          │
│        "estimated_time_hours": 5,                                                                               │
│        "required_resources": ["Bob Smith"]                                                                      │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Responsive Design Mockups",                                                                │
│        "estimated_time_hours": 7,                                                                               │
│        "required_resources": ["Bob Smith"]                                                                      │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Client Review & Approval of Designs",                                                      │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["John Doe", "Bob Smith"]                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Front-End Development (Homepage, S

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 942ef022-d67f-4058-a93c-bd84c1f4bd3e                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2fd32f5c-401e-41b0-aa84-a67f1da993f4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```json                                                                                          │
│  {                                                                                                              │
│    "tasks": [                                                                                                   │
│      {                                                                                                          │
│        "task_name": "Requirements Gathering & Confirmation",                                                    │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["John Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Project Plan Finalization",                                                                │
│        "estimated_time_hours": 1,                                                                               │
│        "required_resources": ["John Doe"]                                                                       │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Wireframing & Mockups (Desktop)",                                                          │
│        "estimated_time_hours": 5,                                                                               │
│        "required_resources": ["Bob Smith"]                                                                      │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Responsive Design Mockups",                                                                │
│        "estimated_time_hours": 7,                                                                               │
│        "required_resources": ["Bob Smith"]                                                                      │
│      },                                                                                                         │
│      {                                                                                                          │
│        "task_name": "Client Review & Approval of Designs",                                                      │
│        "estimated_time_hours": 2,                                                                               │
│        "required_resources": ["John Doe", "Bob Smith"]                                                          │
│      },                                                                                                         │
│      {                                                

In [23]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0013


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,8933,5144,0,3789,3


In [24]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Requirements Gathering & Confirmation',
   'estimated_time_hours': 2.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Project Plan Finalization',
   'estimated_time_hours': 1.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Wireframing & Mockups (Desktop)',
   'estimated_time_hours': 5.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Responsive Design Mockups',
   'estimated_time_hours': 7.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Client Review & Approval of Designs',
   'estimated_time_hours': 2.0,
   'required_resources': ['John Doe', 'Bob Smith']},
  {'task_name': 'Front-End Development (Homepage, Services, About Us, Contact Us)',
   'estimated_time_hours': 15.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Blog Section Development',
   'estimated_time_hours': 7.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'CMS Integration & Configuration',
   'estimated_time_hours': 5.0,
   'req

In [25]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Requirements Gathering & Confirmation,2.000000,['John Doe']
1,Project Plan Finalization,1.000000,['John Doe']
2,Wireframing & Mockups (Desktop),5.000000,['Bob Smith']
3,Responsive Design Mockups,7.000000,['Bob Smith']
4,Client Review & Approval of Designs,2.000000,"['John Doe', 'Bob Smith']"
5,"Front-End Development (Homepage, Services, About Us, Contact Us)",15.000000,['Jane Doe']
6,Blog Section Development,7.000000,['Jane Doe']
7,CMS Integration & Configuration,5.000000,['Jane Doe']
8,Functional Testing,5.000000,"['Alice Johnson', 'Tom Brown']"
9,Performance Testing & SEO Optimization,3.000000,['Jane Doe']


In [26]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Project Initiation & Planning Complete,"['Requirements Gathering & Confirmation', 'Project Plan Finalization']"
1,Design Phase Complete,"['Wireframing & Mockups (Desktop)', 'Responsive Design Mockups', 'Client Review & Approval of Designs']"
2,Development Phase Complete,"['Front-End Development (Homepage, Services, About Us, Contact Us)', 'Blog Section Development', 'CMS Integration & Configuration']"
3,Testing & Deployment Complete,"['Functional Testing', 'Performance Testing & SEO Optimization', 'User Acceptance Testing (UAT)']"
4,Project Closure,['Final Documentation & Handover']
